In [ ]:
# from ccv:


# coding: utf-8

import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
import pickle
from sklearn.cluster import KMeans
from skimage.measure import grid_points_in_poly
from scipy.interpolate import splprep, splrep, splev
from pydescriptors.compactness import hz as compactness_hz
from scipy.ndimage.measurements import center_of_mass
from scipy.optimize import curve_fit
from scipy.ndimage.interpolation import zoom
from nifti import *
import nifti.clib as ncl

def func(x, C, D): 
    return C*x**D

buds = ['sample41_batch11_40_41_DE_stack_ok','sample36_batch8_FGH_stack_ok','sample23_batch4_EF_stack_ok',\
        'sample17_batch3_IJ_stack_ok',\
        'sample09_batch2_stack_ok','sample21_batch4_CD_stack_ok','sample10_batch2_stack_ok',\
        'sample06_batch2_stack_ok','sample34_batch8_IJLK_stack_ok','sample38_batch10_BCD_stack_ok',\
        'sample13_batch3_B_stack_m??ig','sample44_batch10_EFG_stack_ok','sample01_batch1_stack_cropped_ok',\
        'sample15_batch5_DE_stack_m??ig','sample08_batch2_stack','sample19_batch3_EFGH_stack_ok',\
        'sample20_batch4_AB_stack_ok','T2', 'T3', 'Viburnum_batch12_sampleT6_redo_cropped']

for d in buds:
    # open image files
    print d
    print '   open and cluster images...'

    files = []
    for file in glob.glob('image_stacks/'+d+'/*.ti*'):
        files.append(file)

    files.sort()
    
    scale = False    
    im = np.array(Image.open(files[0]))
    if np.any(np.array([len(files),np.shape(im)[0],np.shape(im)[1]])>1000):
        print '   DOWNSCALING imagestack by a factor of 2!'
        scale = True
    
    if scale:
        im = zoom(im,0.5)
        image_stack = np.zeros([len(files)/2+1,np.shape(im)[0],np.shape(im)[1]])
        for i in range(len(files)):
            if i%2 == 0:
                im = zoom(np.array(Image.open(files[i])),0.5)
                image_stack[i/2,:,:] = im
    else:
        image_stack = np.zeros([len(files),np.shape(im)[0],np.shape(im)[1]])
        for i in range(len(files)):
            im = np.array(Image.open(files[i]))
            image_stack[i,:,:] = im

    # do the clustering on the whole image stack 
    cls = KMeans(2).fit_predict(image_stack.reshape([np.prod(np.shape(image_stack)),1]))            
    # make sure the background is 0, the bud is 1, clustering randomizes which one is which
    # assuming here that the corner pixel is always background
    if cls[0] != 0:
        cls ^= 1
    bud_voxels = cls.reshape(np.shape(image_stack))
        
    print np.shape(image_stack),np.shape(bud_voxels)
    print len(files),np.shape(im)[0],np.shape(im)[1]
    print '   done'

    # find all leaf voxels
    leaf_voxels = np.zeros(np.shape(image_stack))

    print '   find leaf voxels...'

    # load the clicks
    file_name = 'data_files/'+d+'_shell_leaf_boundaries_d1.dat'
    f = open(file_name,'r')
    [shell_leaf,fs] = pickle.load(f)
    f.close()

    indx = np.array([i for i, ltr in enumerate(fs[0]) if ltr == '.'])
    frames = np.array([int(f[indx[-1]-4:indx[-1]]) for f in fs])
    if scale:
        frames = frames/2
    print frames
    # do the periodic spline interpolation in the x-y frame and collect 1000 points on each frame
    points = np.zeros([len(frames),3,1000])
    for f in range(len(frames)):

        boundary = np.array(shell_leaf[f])

        if len(boundary)>0:
            if scale:
                boundary = boundary / 2e0
                
            center = [np.sum(boundary[:,0])/len(boundary[:,0]),np.sum(boundary[:,1])/len(boundary[:,1])]

            tck, u = splprep(boundary.T, u=None, s=0.0, per=1) 

            u_new = np.linspace(u.min(), u.max(), 1000)
            x_new, y_new = splev(u_new, tck, der=0)

            phi = np.arctan2(y_new-center[1], x_new-center[0])/np.pi
            indx = np.where(phi == np.max(phi))[0]

            x_shift = np.append(x_new[indx:],x_new[:indx])
            y_shift = np.append(y_new[indx:],y_new[:indx])
            z_shift = np.zeros(len(x_new))+frames[f]

            points[f] = np.array([x_shift,y_shift,z_shift])    

    # do the vertical interpolation and collect the tcks for splev
    splev_x = []
    splev_y = []
    for i in range(1000):
        x = points[:,0,i]
        y = points[:,1,i]
        z = points[:,2,i]
        tckx = splrep(z[x>0],x[x>0])
        splev_x.append(tckx)

        tcky = splrep(z[x>0],y[x>0])
        splev_y.append(tcky)

    # go through the frames and plot the interpolated contours

    plot = False

    alpha = 0.6
    colors = [(0.0, 0.0, 0.0, 0e0),  # black
                (1.0, 0.0, 0.0, alpha)]  # red
    cmap1 = matplotlib.colors.ListedColormap(colors)
    colors = [(0.0, 0.0, 0.0, 0e0),  # black
                (0.0, 0.0, 1.0, alpha)]  # blue
    cmap2 = matplotlib.colors.ListedColormap(colors)
    
    # check in between which frames should the leaf voxels be found
    length = np.zeros(len(frames))
    for f in range(len(frames)):
        length[f] = len(np.array(shell_leaf[f]))

    i_min = frames[np.min(np.where(length>0)[0])]
    i_max = frames[np.max(np.where(length>0)[0])]
    
    f = 0
    for i in range(np.shape(image_stack)[0]):
        if plot:
            plt.figure(figsize=(6,6))
            plt.axis('equal')
            plt.title('blue: scale, red: leaf, uncolored: background')
            plt.xlabel('pixel coord.')
            plt.ylabel('pixel coord.')
            plt.imshow(image_stack[i,:,:].T,cmap='Greys_r',vmin=10000,vmax=33000)
        if (i == frames[f+1]) and (f < len(frames)-2):
            f = f + 1
        boundary = np.array(shell_leaf[f])
        if (i >= i_min) and (i <= i_max):
            if scale:
                boundary = boundary / 2e0
            x_cont = np.zeros(1000)
            y_cont = np.zeros(1000)
            for j in range(1000):
                tck_x = splev_x[j]
                tck_y = splev_y[j]
                x_cont[j] = splev(i,tck_x)
                y_cont[j] = splev(i,tck_y)
            pixels_in_cont = grid_points_in_poly(np.shape(bud_voxels[i,:,:]),np.array([x_cont,y_cont]).T)
            pixels_in_cont = np.rint(pixels_in_cont == True)
            leaf_voxels[i,:,:] = bud_voxels[i,:,:]*pixels_in_cont

            if plot: 
                plt.imshow(leaf_voxels[i,:,:].T,cmap=cmap1)
                plt.imshow((bud_voxels[i,:,:]-leaf_voxels[i,:,:]).T,cmap=cmap2)

                if i == frames[f]-1:
                    plt.plot(boundary[:,0],boundary[:,1],'wo')
                if i == frames[f]:
                    plt.plot(boundary[:,0],boundary[:,1],'wo')
                if i == frames[f]+1:
                    plt.plot(boundary[:,0],boundary[:,1],'wo')
        else:
            if plot:
                plt.imshow(bud_voxels[i,:,:].T,cmap=cmap2)

        if plot:
            plt.xlim([0,np.shape(image_stack[i,:,:])[0]])
            plt.ylim([0,np.shape(image_stack[i,:,:])[1]])    
            plt.savefig('animation_border/interpolate/frame'+str(i).zfill(4)+'.png',dpi=100)
            plt.close()
            
    # pickle dump the arrays
    #file_name = 'data_files/segmented_'+d+'.dat'
    #f = open(file_name,'wb')
    #pickle.dump([bud_voxels+leaf_voxels],f)
    #f.close()

    nim = NiftiImage(leaf_voxels)
    nim.header['datatype'] == ncl.NIFTI_TYPE_FLOAT64
    nim.save('data_files/segmented_leaf_'+d+'.nii.gz')
    
    nim = NiftiImage(bud_voxels+leaf_voxels)
    print nim.header['dim']
    nim.header['datatype'] == ncl.NIFTI_TYPE_FLOAT64
    nim.save('data_files/segmented_'+d+'.nii.gz')

    print '   done'
    

    # calculate features

    print '   calculating features...'

    # nr of bud voxels
    nr_bud_vox = np.sum(bud_voxels)

    # leaf to bud volume ratio
    lf_to_bud = np.sum(leaf_voxels)/nr_bud_vox

    # compactness
    X,Y,Z = leaf_voxels.nonzero()
    comp = compactness_hz(X,Y,Z)

    # fractal dimension and normalized counts
    CoM = center_of_mass(leaf_voxels)
    #print CoM
    #print np.any(leaf_voxels == 1)
    #print np.shape(np.where(leaf_voxels == 1)[0])
    #print np.where(leaf_voxels == 1)
    #print leaf_voxels
    scale = 2e0

    nr_radii = int(np.max(np.shape(leaf_voxels))/scale)
    radii = np.arange(nr_radii+1)*scale
    x,y,z = np.ogrid[-CoM[0]:np.shape(leaf_voxels)[0]-CoM[0], -CoM[1]:np.shape(leaf_voxels)[1]-CoM[1],-CoM[2]:np.shape(leaf_voxels)[2]-CoM[2]]

    count = np.zeros(len(radii))

    for i in range(nr_radii):
        mask_outer = x*x + y*y + z*z <= radii[i+1]**2e0
        count[i] = np.sum(leaf_voxels[mask_outer])

    if np.max(count) != np.sum(leaf_voxels):
        print 'problem with shell histo'
        print np.max(count), np.sum(leaf_voxels)
        raise ValueError

    max_rad = np.argmax(radii[count < np.max(count)])

    # fit power law
    #[C,D], pcov = curve_fit(func, radii[1:max_rad/3], count[1:max_rad/3],bounds = ([0e0,1e0],[np.inf,3e0]))
    [C,D], pcov = curve_fit(func, radii[1:max_rad/3], count[1:max_rad/3])

    perr = np.sqrt(np.diag(pcov))
    
    file_name = 'data_files/shell_hist_'+d+'.dat'
    f = open(file_name,'wb')
    pickle.dump([count,radii],f)
    f.close()
    
    print d,lf_to_bud,nr_bud_vox,comp,D,perr[1],count[radii[1:] == radii[int(max_rad/4e0)]]/np.max(count),\
        count[radii[1:] == radii[int(max_rad/2e0)]]/np.max(count),count[radii[1:] == radii[int(3e0*max_rad/4e0)]]/np.max(count)

sample34_batch8_IJLK_stack_ok
   open and cluster images...
   DOWNSCALING imagestack by a factor of 2!


In [1]:
# from ccv:


# coding: utf-8

import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
import pickle
from sklearn.cluster import KMeans
from skimage.measure import grid_points_in_poly
from scipy.interpolate import splprep, splrep, splev
from pydescriptors.compactness import hz as compactness_hz
from scipy.ndimage.measurements import center_of_mass
from scipy.optimize import curve_fit
from scipy.ndimage.interpolation import zoom
from nifti import *
import nifti.clib as ncl

def func(x, C, D): 
    return C*x**D

buds = ['Viburnum_batch12_sampleT6_redo_cropped','sample01_batch1_stack_cropped_ok']

for d in buds:
    # open image files
    print d
    print '   open and cluster images...'

    files = []
    for file in glob.glob('image_stacks/'+d+'/*.ti*'):
        files.append(file)

    files.sort()
    
    scale = False    
    im = np.array(Image.open(files[0]))
    if np.any(np.array([len(files),np.shape(im)[0],np.shape(im)[1]])>1000):
        print '   DOWNSCALING imagestack by a factor of 4!'
        scale = True

    print len(files)
    print '   find leaf voxels...'

    # load the clicks
    file_name = 'data_files/'+d+'_shell_leaf_boundaries_d1.dat'
    f = open(file_name,'r')
    [shell_leaf,fs] = pickle.load(f)
    f.close()

    indx = np.array([i for i, ltr in enumerate(fs[0]) if ltr == '.'])
    frames = np.array([int(f[indx[-1]-4:indx[-1]]) for f in fs])
    if scale:
        frames = frames/4
    print frames

    if scale:
        im = zoom(im,0.25)
        image_stack = np.zeros([len(files)/4+1,np.shape(im)[0],np.shape(im)[1]])
        for i in range(len(files)):
            if i%4 == 0:
                im = zoom(np.array(Image.open(files[i])),0.25)
                image_stack[i/4,:,:] = im
    else:
        image_stack = np.zeros([len(files),np.shape(im)[0],np.shape(im)[1]])
        for i in range(len(files)):
            im = np.array(Image.open(files[i]))
            image_stack[i,:,:] = im

    # do the clustering on the whole image stack 
    cls = KMeans(2).fit_predict(image_stack.reshape([np.prod(np.shape(image_stack)),1]))            
    # make sure the background is 0, the bud is 1, clustering randomizes which one is which
    # assuming here that the corner pixel is always background
    if cls[0] != 0:
        cls ^= 1
    bud_voxels = cls.reshape(np.shape(image_stack))
        
    print np.unique(bud_voxels,return_counts=True)
    
    nim = NiftiImage(bud_voxels)
    nim.header['datatype'] == ncl.NIFTI_TYPE_FLOAT64
    nim.save('data_files/segmented_bud1_'+d+'.nii.gz')
    
    print '   done'

    # find all leaf voxels
    leaf_voxels = np.zeros(np.shape(image_stack))

    # do the periodic spline interpolation in the x-y frame and collect 1000 points on each frame
    points = np.zeros([len(frames),3,1000])
    for f in range(len(frames)):

        boundary = np.array(shell_leaf[f])

        if len(boundary)>0:
            if scale:
                boundary = boundary / 4e0
                
            center = [np.sum(boundary[:,0])/len(boundary[:,0]),np.sum(boundary[:,1])/len(boundary[:,1])]

            tck, u = splprep(boundary.T, u=None, s=0.0, per=1) 

            u_new = np.linspace(u.min(), u.max(), 1000)
            x_new, y_new = splev(u_new, tck, der=0)

            phi = np.arctan2(y_new-center[1], x_new-center[0])/np.pi
            indx = np.where(phi == np.max(phi))[0]

            x_shift = np.append(x_new[indx:],x_new[:indx])
            y_shift = np.append(y_new[indx:],y_new[:indx])
            z_shift = np.zeros(len(x_new))+frames[f]

            points[f] = np.array([x_shift,y_shift,z_shift])    

    # do the vertical interpolation and collect the tcks for splev
    splev_x = []
    splev_y = []
    for i in range(1000):
        x = points[:,0,i]
        y = points[:,1,i]
        z = points[:,2,i]
        tckx = splrep(z[x>0],x[x>0])
        splev_x.append(tckx)

        tcky = splrep(z[x>0],y[x>0])
        splev_y.append(tcky)

    # go through the frames and plot the interpolated contours
    print np.unique(bud_voxels,return_counts=True)
    nim = NiftiImage(bud_voxels)
    nim.header['datatype'] == ncl.NIFTI_TYPE_FLOAT64
    nim.save('data_files/segmented_bud2_'+d+'.nii.gz')

    plot = True

    alpha = 0.6
    colors = [(0.0, 0.0, 0.0, 0e0),  # black
                (1.0, 0.0, 0.0, alpha)]  # red
    cmap1 = matplotlib.colors.ListedColormap(colors)
    colors = [(0.0, 0.0, 0.0, 0e0),  # black
                (0.0, 0.0, 1.0, alpha)]  # blue
    cmap2 = matplotlib.colors.ListedColormap(colors)
    
    # check in between which frames should the leaf voxels be found
    length = np.zeros(len(frames))
    for f in range(len(frames)):
        length[f] = len(np.array(shell_leaf[f]))

    i_min = frames[np.min(np.where(length>0)[0])]
    i_max = frames[np.max(np.where(length>0)[0])]
    
    f = 0
    for i in range(np.shape(image_stack)[0]):
        if plot:
            plt.figure(figsize=(6,6))
            plt.axis('equal')
            plt.title('blue: scale, red: leaf, uncolored: background')
            plt.xlabel('pixel coord.')
            plt.ylabel('pixel coord.')
            plt.imshow(image_stack[i,:,:].T,cmap='Greys_r',vmin=10000,vmax=33000)
        if (i == frames[f+1]) and (f < len(frames)-2):
            f = f + 1
        boundary = np.array(shell_leaf[f])
        if (i >= i_min) and (i <= i_max):
            if scale:
                boundary = boundary / 4e0
            x_cont = np.zeros(1000)
            y_cont = np.zeros(1000)
            for j in range(1000):
                tck_x = splev_x[j]
                tck_y = splev_y[j]
                x_cont[j] = splev(i,tck_x)
                y_cont[j] = splev(i,tck_y)
            pixels_in_cont = grid_points_in_poly(np.shape(bud_voxels[i,:,:]),np.array([x_cont,y_cont]).T)
            pixels_in_cont = np.rint(pixels_in_cont == True)
            leaf_voxels[i,:,:] = bud_voxels[i,:,:]*pixels_in_cont
            
            if plot: 
                plt.imshow(leaf_voxels[i,:,:].T,cmap=cmap1)
                plt.imshow((bud_voxels[i,:,:]-leaf_voxels[i,:,:]).T,cmap=cmap2)

                if i == frames[f]-1:
                    plt.plot(boundary[:,0],boundary[:,1],'wo')
                if i == frames[f]:
                    plt.plot(boundary[:,0],boundary[:,1],'wo')
                if i == frames[f]+1:
                    plt.plot(boundary[:,0],boundary[:,1],'wo')
        else:
            if plot:
                plt.imshow(bud_voxels[i,:,:].T,cmap=cmap2)

        if plot:
            plt.xlim([0,np.shape(image_stack[i,:,:])[0]])
            plt.ylim([0,np.shape(image_stack[i,:,:])[1]])    
            plt.savefig('animation_border/interpolate/frame_'+d+'_'+str(i).zfill(4)+'.png',dpi=100)
            plt.close()
            
    print np.unique(bud_voxels,return_counts=True)

    print np.unique(leaf_voxels,return_counts=True)

    nim = NiftiImage(bud_voxels)
    nim.header['datatype'] == ncl.NIFTI_TYPE_FLOAT64
    nim.save('data_files/segmented_bud3_'+d+'.nii.gz')

    nim = NiftiImage(bud_voxels+leaf_voxels)
    print nim.header['dim']
    nim.header['datatype'] == ncl.NIFTI_TYPE_FLOAT64
    nim.save('data_files/segmented_'+d+'.nii.gz')

    print '   done'
    
    # calculate features

    print '   calculating features...'

    # nr of bud voxels
    nr_bud_vox = np.sum(bud_voxels)

    # leaf to bud volume ratio
    lf_to_bud = np.sum(leaf_voxels)/nr_bud_vox

    # compactness
    X,Y,Z = leaf_voxels.nonzero()
    comp = compactness_hz(X,Y,Z)

    # fractal dimension and normalized counts
    CoM = center_of_mass(leaf_voxels)
    #print CoM
    #print np.any(leaf_voxels == 1)
    #print np.shape(np.where(leaf_voxels == 1)[0])
    #print np.where(leaf_voxels == 1)
    #print leaf_voxels
    scale = 2e0

    nr_radii = int(np.max(np.shape(leaf_voxels))/scale)
    radii = np.arange(nr_radii+1)*scale
    x,y,z = np.ogrid[-CoM[0]:np.shape(leaf_voxels)[0]-CoM[0], -CoM[1]:np.shape(leaf_voxels)[1]-CoM[1],-CoM[2]:np.shape(leaf_voxels)[2]-CoM[2]]

    count = np.zeros(len(radii))

    for i in range(nr_radii):
        mask_outer = x*x + y*y + z*z <= radii[i+1]**2e0
        count[i] = np.sum(leaf_voxels[mask_outer])

    if np.max(count) != np.sum(leaf_voxels):
        print 'problem with shell histo'
        print np.max(count), np.sum(leaf_voxels)
        raise ValueError

    max_rad = np.argmax(radii[count < np.max(count)])

    # fit power law
    #[C,D], pcov = curve_fit(func, radii[1:max_rad/3], count[1:max_rad/3],bounds = ([0e0,1e0],[np.inf,3e0]))
    [C,D], pcov = curve_fit(func, radii[1:max_rad/3], count[1:max_rad/3])

    perr = np.sqrt(np.diag(pcov))
    
    file_name = 'data_files/shell_hist_'+d+'.dat'
    f = open(file_name,'wb')
    pickle.dump([count,radii],f)
    f.close()
    
    print d,lf_to_bud,nr_bud_vox,comp,D,perr[1],count[radii[1:] == radii[int(max_rad/4e0)]]/np.max(count),\
        count[radii[1:] == radii[int(max_rad/2e0)]]/np.max(count),count[radii[1:] == radii[int(3e0*max_rad/4e0)]]/np.max(count)

Viburnum_batch12_sampleT6_redo_cropped
   open and cluster images...
   DOWNSCALING imagestack by a factor of 4!
2747
   find leaf voxels...
[  0  17  34  51  68  85 103 120 137 154 171 188 206 223 240 257 274 291
 309 326 343 360 377 394 412 429 446 463 480 497 515 532 549 566 583 600
 618 635 652 669]


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/ndimage/interpolation.py:549: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


(array([0, 1], dtype=int32), array([88470388, 25519400]))


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/nifti/image.py:231: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return (not self._data == None)


   done


...

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:266: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 344 but corresponding boolean dimension is 343


[  0  18  36  54  72  91 109 127 145 164 182 200 218 237 255 273 291 310
 328 346 364 382 401 419 437 455 474 492 510 528 547 565 583 601 620 638
 656 674 693 711]
(array([0, 1], dtype=int32), array([59803151, 22834309]))
   done


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/interpolate/fitpack.py:227: RuntimeWarning: Setting x[0][13]=x[0][0]
  (i, m, i)))
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/interpolate/fitpack.py:227: RuntimeWarning: Setting x[1][13]=x[1][0]
  (i, m, i)))
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/interpolate/fitpack.py:227: RuntimeWarning: Setting x[0][14]=x[0][0]
  (i, m, i)))
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/interpolate/fitpack.py:227: RuntimeWarning: Setting x[1][14]=x[1][0]
  (i, m, i)))
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/interpolate/fitpack.py:227: RuntimeWarning: Setting x[0][15]=x[0][0]
  (i, m, i)))
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/interpolate/fitpack.p

(array([0, 1], dtype=int32), array([59803151, 22834309]))
(array([0, 1], dtype=int32), array([59803151, 22834309]))
(array([ 0.,  1.]), array([76244707,  6392753]))
[3L, 342L, 331L, 730L, 1L, 1L, 1L, 1L]
   done
   calculating features...
sample01_batch1_stack_cropped_ok 0.279962621159 22834309 0.509734585631 2.4692491282 0.0234253648745 [ 0.25086829] [ 0.81943006] [ 0.96585118]


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:266: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 366 but corresponding boolean dimension is 365


In [10]:
import os
import numpy as np
from nifti import *
import nifti.clib as ncl

# collect segmentations
segs = []
for file in os.listdir("data_files/"):
    if file.endswith(".nii.gz") and ('segmented' in file):
        segs.append(file)
        
for s in segs:
    nim = NiftiImage('data_files/'+s).asarray()
    print s,np.sum(nim > 0),np.sum(nim > 1)

 segmented_sample01_batch1_stack_cropped_ok.nii.gz 19073870 6392753
segmented_sample06_batch2_stack_ok.nii.gz 17441956 3406082
segmented_sample08_batch2_stack.nii.gz 2034504 2031034
segmented_sample09_batch2_stack_ok.nii.gz 13517311 2711576
segmented_sample10_batch2_stack_ok.nii.gz 18650032 2930484
segmented_sample13_batch3_B_stack_m??ig.nii.gz 84241125 19100564
segmented_sample15_batch5_DE_stack_m??ig.nii.gz 9570903 9498801
segmented_sample17_batch3_IJ_stack_ok.nii.gz 17739874 4278795
segmented_sample19_batch3_EFGH_stack_ok.nii.gz 25091124 24544268
segmented_sample20_batch4_AB_stack_ok.nii.gz 11426974 2122325
segmented_sample21_batch4_CD_stack_ok.nii.gz 8709085 3720297
segmented_sample23_batch4_EF_stack_ok.nii.gz 11291449 3424017
segmented_sample34_batch8_IJLK_stack_ok.nii.gz 20644288 20492529
segmented_sample36_batch8_FGH_stack_ok.nii.gz 33692124 6164648
segmented_sample38_batch10_BCD_stack_ok.nii.gz 2692983 2681322
segmented_sample41_batch11_40_41_DE_stack_ok.nii.gz 12038786 4154339